# (16.0) Описание решаемой задачи

В окружении выявленного айсберга необходимо осуществить поиск заявок, которые потенциально могут быть элементами цепочки всплытия айсберга, но при исполнении не были выявлены как айсберг, так как сделка была произведена ровно на объем видимой части

# (16.1) Установка всех необходимых пакетов¶

In [1]:
import numpy as np
import pandas as pd
import datetime as dt

# (16.2) Определение функций¶

In [2]:
# перевод времени в формат datetime
def ReverseToDateTime(t):
    format = '%Y%m%d%H%M%S%f'
    time_str = dt.datetime.strptime(t,format)
    return time_str

# восстановление стакана на момент идентификации айсберга
def GlassBuild(for_lob):
    # устанавливаем маску лимитных заявок заявках (True - лимитная заявка)
    no_mkt_orders = for_lob['PRICE']!=0
    # таблица выставленных заявок
    plc_orders = for_lob[['ORDERNO','BUYSELL','PRICE','VOLUME']][(for_lob ['ACTION']==1) & no_mkt_orders]
    # таблица отмененных заявок
    wdr_orders = for_lob[['ORDERNO','VOLUME']][(for_lob ['ACTION'] == 0) & no_mkt_orders]
    # таблица исполненных заявок
    trd_orders = for_lob [['ORDERNO','VOLUME']][(for_lob['ACTION']==2) & no_mkt_orders]
    trd_orders = trd_orders.groupby(['ORDERNO']).sum()
    trd_orders['ORDERNO'] = trd_orders.index
    trd_orders.index=range(trd_orders.shape[0])
    # собираем все типы заявок в один датафрейм - очередь заявок
    lob_almost = pd.merge(plc_orders,wdr_orders, on='ORDERNO',how='left',suffixes=('_plc','_wdr'))
    lob_almost = pd.merge(lob_almost,trd_orders, on='ORDERNO', how='left')
    lob_almost = lob_almost.rename(columns={'VOLUME':'VOLUME_trd'})
    lob_almost = lob_almost.fillna(0)
    lob_almost['TOTAL_VOL'] = lob_almost['VOLUME_plc'] - lob_almost['VOLUME_wdr'] - lob_almost['VOLUME_trd']
    # удаляем из очереди заявок заявки с нулевым или отрицательным объемом
    lob_almost = lob_almost[['ORDERNO','BUYSELL','PRICE','VOLUME_plc','VOLUME_wdr','VOLUME_trd','TOTAL_VOL']][(lob_almost['TOTAL_VOL']>0)]
    # строим стакан
    lob = lob_almost.pivot_table(index = "PRICE", columns = "BUYSELL", values = "TOTAL_VOL", aggfunc = 'sum')
    lob.sort_index(axis = 0, inplace = True, ascending = False)
    lob = lob.fillna(0)
    # расчет лучшей цены на покупку
    bid_price = lob[lob['B'] > 0].index[0]
    # расчет объема на уровне лучшей цены на покупку
    bid_volume = lob['B'][bid_price]
    # расчет лучшей цены на продажу
    ask_price = lob[lob['S'] > 0].index[-1]
    # расчет объема на уровне лучшей цены на продажу
    ask_volume = lob['S'][ask_price]
    
    liquidity = [bid_price, bid_volume, ask_price, ask_volume]
     
    return(liquidity)

'''def GlassUpdate(lob_almost):
    
    
def GlassBuild(lob_almost):
    

def lob_almost(orders):'''

'def GlassUpdate(lob_almost):\n    \n    \ndef GlassBuild(lob_almost):\n    \n\ndef lob_almost(orders):'

# (16.3) Список всех используемых датафреймов

In [ ]:
# orders - ордерлог на заданную дату
# iceberg_all - список всех найденных айсбергов
# orders_instr - ордерлог по инструменту

# (16.4) Чтение данных¶

In [128]:
%%time
# преобразуем time в формат datetime
# orders - ордерлог за день
orderlog_name = 'OrderLog'+'20190304'+'.txt'
orders = pd.read_csv(orderlog_name, header = 0)
orders['together_time'] = str(date)+orders['TIME'].apply(str)
orders['TIME'] = [ReverseToDateTime(orders.together_time[i]) for i in range (len(orders.together_time))]
del orders['together_time']

CPU times: user 5min 20s, sys: 2.27 s, total: 5min 22s
Wall time: 5min 23s


In [3]:
# создаем список из всех названий файлов с данными (из них в дальнейшем можно извлечь дату)
orderlogs = ['OrderLog20190304.txt',
             'OrderLog20190305.txt',
             'OrderLog20190306.txt',
             'OrderLog20190307.txt',
             'OrderLog20190311.txt',
             'OrderLog20190312.txt',
             'OrderLog20190313.txt',
             'OrderLog20190314.txt',
             'OrderLog20190315.txt',
             'OrderLog20190318.txt',
             'OrderLog20190319.txt',
             'OrderLog20190320.txt',
             'OrderLog20190321.txt',
             'OrderLog20190322.txt',
             'OrderLog20190325.txt',
             'OrderLog20190326.txt',
             'OrderLog20190327.txt',
             'OrderLog20190328.txt',
             'OrderLog20190329.txt'
            ]

In [4]:
# создаем список из инструментов
tickers = ['GAZP',
           'SBER',
           'GMKN',
           'LKOH',
           'MTSS',
           'MGNT',
           'TATN',
           'NVTK',
           'YNDX',
           'ROSN',
           'FIVE',
           'VTBR',
           'SNGS',
           'CHMF',
           'ALRS'
          ]

In [124]:
# читаем список выявленных айсберг-заявок + преобразуем время в формат datetime
# iceberg_all - список всех найденных айсбергов
iceberg_all = pd.read_csv('iceberg.csv', header = 0)
iceberg_all['together_time'] = iceberg_all['DATE'].apply(str)+iceberg_all['TIME'].apply(str)
iceberg_all['together_time_identified'] = iceberg_all['DATE'].apply(str)+iceberg_all['TIME_IDENT'].apply(str)
iceberg_all['TIME'] = [ReverseToDateTime(iceberg_all.together_time[i]) for i in range (len(iceberg_all.together_time))]
iceberg_all['TIME_IDENT'] = [ReverseToDateTime(iceberg_all.together_time_identified[i]) for i in range (len(iceberg_all.together_time_identified))]
iceberg_all['DELTA'] = iceberg_all['TIME_IDENT'] - iceberg_all['TIME']
del iceberg_all['together_time']
del iceberg_all['together_time_identified']

iceberg_all

,index,BUYSELL,DATE,ICEBERG,ORDERNO,PRICE,SECCODE,TIME,TIME_IDENT,VOLUME,VOLUME_INI,DELTA
0,0,B,20190304,1,199077,156.28,GAZP,2019-03-04 10:06:43.595004,2019-03-04 10:09:36.467866,-2830,2000,00:02:52.872862
1,1,B,20190304,1,359638,156.18,GAZP,2019-03-04 10:30:27.072289,2019-03-04 10:53:31.759856,-210,2000,00:23:04.687567
2,2,S,20190304,1,397852,156.58,GAZP,2019-03-04 10:36:57.193855,2019-03-04 10:38:05.393784,-3000,2000,00:01:08.199929
3,3,B,20190304,1,486370,156.18,GAZP,2019-03-04 10:53:31.809856,2019-03-04 10:53:42.904115,-1000,1790,00:00:11.094259
4,4,B,20190304,1,516105,156.03,GAZP,2019-03-04 11:00:03.906212,2019-03-04 11:00:05.256841,-1000,2000,00:00:01.350629
...,...,...,...,...,...,...,...,...,...,...,...,...
54501,436,B,20190329,1,3821455,92.47,ALRS,2019-03-29 18:32:19.963550,2019-03-29 18:32:19.963586,-360,930,00:00:00.000036
54502,437,B,20190329,1,3845898,92.58,ALRS,2019-03-29 18:35:14.255473,2019-03-29 18:35:14.256529,-1000,1000,00:00:00.001056
54503,438,B,20190329,1,3845899,92.58,ALRS,2019-03-29 18:35:14.255473,2019-03-29 18:35:14.256529,-490,1000,00:00:00.001056
54504,439,B,20190329,1,3845904,92.58,ALRS,2019-03-29 18:35:14.257134,2019-03-29 18:35:14.257134,-770,510,00:00:00


In [145]:
iceberg_all[(iceberg_all.SECCODE == 'SBER') & (iceberg_all.DATE == 20190304)]

,index,BUYSELL,DATE,ICEBERG,ORDERNO,PRICE,SECCODE,TIME,TIME_IDENT,VOLUME,VOLUME_INI,DELTA
174,0,B,20190304,1,47392,206.99,SBER,2019-03-04 10:00:03.463857,2019-03-04 10:00:03.463857,-920,1000,00:00:00
175,1,B,20190304,1,47393,206.99,SBER,2019-03-04 10:00:03.463857,2019-03-04 10:00:03.463857,-120,80,00:00:00
176,2,B,20190304,1,47394,206.99,SBER,2019-03-04 10:00:03.463857,2019-03-04 10:00:03.463857,-2630,880,00:00:00
177,3,B,20190304,1,47395,206.99,SBER,2019-03-04 10:00:03.463857,2019-03-04 10:00:03.463857,-1130,370,00:00:00
178,4,B,20190304,1,47396,206.99,SBER,2019-03-04 10:00:03.463857,2019-03-04 10:00:03.463857,-70,870,00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
989,815,B,20190304,1,2454926,205.50,SBER,2019-03-04 18:13:52.680839,2019-03-04 18:13:54.820780,-820,80,00:00:02.139941
990,816,B,20190304,1,2455091,205.50,SBER,2019-03-04 18:13:54.878553,2019-03-04 18:13:58.463577,-11680,180,00:00:03.585024
991,817,B,20190304,1,2576329,205.70,SBER,2019-03-04 18:38:56.417597,2019-03-04 18:38:57.219243,-49000,30000,00:00:00.801646
992,818,B,20190304,1,2576487,205.70,SBER,2019-03-04 18:38:57.219797,2019-03-04 18:38:57.528890,-1000,11000,00:00:00.309093


In [10]:
iceberg_all.dtypes

index                   int64
BUYSELL                object
DATE                    int64
ICEBERG                 int64
ORDERNO                 int64
PRICE                 float64
SECCODE                object
TIME           datetime64[ns]
TIME_IDENT     datetime64[ns]
VOLUME                  int64
VOLUME_INI              int64
DELTA         timedelta64[ns]
dtype: object

In [181]:
# выбираем один айсберг для анализа его окружения и извлекаем основную информацию из него
id = 174
test = iceberg_all.loc[id]
buysell = iceberg_all.loc[id][1]
date = iceberg_all.loc[id][2]
orderno = iceberg_all.loc[id][4]
price = iceberg_all.loc[id][5]
ticker = iceberg_all.loc[id][6]
time_place = iceberg_all.loc[id][7]
time_ident = iceberg_all.loc[id][8]
volume = iceberg_all.loc[id][10]

test

index                                  0
BUYSELL                                B
DATE                            20190304
ICEBERG                                1
ORDERNO                            47392
PRICE                             206.99
SECCODE                             SBER
TIME          2019-03-04 10:00:03.463857
TIME_IDENT    2019-03-04 10:00:03.463857
VOLUME                              -920
VOLUME_INI                          1000
DELTA                    0 days 00:00:00
Name: 174, dtype: object

In [178]:
# выбираем из ордерлога все, что инструменту на момент идентификации айсберга + 5 минут после его идентификации
# orders_instr - ордерлог по инструменту
orders_instr = orders[(orders.SECCODE == ticker)].reset_index()
orders_instr

,index,NO,SECCODE,BUYSELL,TIME,ORDERNO,ACTION,PRICE,VOLUME,TRADENO,TRADEPRICE
0,321,322,SBER,S,2019-03-04 10:00:00.000000,322,1,224.15,10,NaN,NaN
1,323,324,SBER,B,2019-03-04 10:00:00.000000,324,1,193.50,10,NaN,NaN
2,328,329,SBER,S,2019-03-04 10:00:00.000000,329,1,217.59,10,NaN,NaN
3,368,369,SBER,B,2019-03-04 10:00:00.000000,369,1,188.56,10,NaN,NaN
4,416,417,SBER,S,2019-03-04 10:00:00.000000,417,1,212.82,100,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
433344,5424213,5424214,SBER,B,2019-03-04 18:44:26.839739,2095784,0,205.31,700,NaN,NaN
433345,5424222,5424223,SBER,B,2019-03-04 18:44:32.933404,606970,0,204.50,1000,NaN,NaN
433346,5424252,5424253,SBER,S,2019-03-04 18:44:44.743345,2492156,0,206.00,10,NaN,NaN
433347,5424277,5424278,SBER,S,2019-03-04 18:44:54.361484,265974,0,210.32,43500,NaN,NaN


# (16.5) Определяем интервал после вскрытия айсберга для поиска потенциальных элементов микроцепочки



In [152]:
# выбираем один айсберг для анализа его окружения и извлекаем основную информацию из него
id = 174
test = iceberg_all.loc[id]
buysell = iceberg_all.loc[id][1]
date = iceberg_all.loc[id][2]
orderno = iceberg_all.loc[id][4]
price = iceberg_all.loc[id][5]
ticker = iceberg_all.loc[id][6]
time_place = iceberg_all.loc[id][7]
time_ident = iceberg_all.loc[id][8]
volume = iceberg_all.loc[id][10]

test

index                                  0
BUYSELL                                B
DATE                            20190304
ICEBERG                                1
ORDERNO                            47392
PRICE                             206.99
SECCODE                             SBER
TIME          2019-03-04 10:00:03.463857
TIME_IDENT    2019-03-04 10:00:03.463857
VOLUME                              -920
VOLUME_INI                          1000
DELTA                    0 days 00:00:00
Name: 174, dtype: object

In [184]:
%%time
ice_index = orders_instr.index[(orders_instr.ORDERNO == orderno)&(orders_instr.ACTION == 2)&(orders_instr.TIME == time_ident)].to_list()[0]
cur_price = orders_instr.loc[ice_index,'PRICE']

for i in range(ice_index, len(orders_instr)):
        
        cur_index = i
        for_lob = orders_instr[orders_instr.index <= cur_index]

        if buysell == 'B':
            
            if GlassBuild(for_lob)[0] > cur_price:

                right_border = orders_instr.loc[cur_index,'TIME']
                right_index = cur_index
                
                break

        elif buysell == 'S':
            
            if GlassBuild(for_lob)[2] < cur_price:
                
                right_border = orders_instr.loc[cur_index,'TIME']
                right_index = cur_index
                 
                break

print(right_border, right_index)

2019-03-04 10:00:03.898113 4243
CPU times: user 7.68 s, sys: 84.2 ms, total: 7.77 s
Wall time: 7.61 s


In [183]:
ice_index = orders_instr.index[(orders_instr.ORDERNO == orderno)&(orders_instr.ACTION == 2)&(orders_instr.TIME == time)].to_list()[0]

IndexError: list index out of range

# (16.6) Определяем интервал перед выставлением айсберга для поиска потенциальных элементов микроцепочки

In [161]:
# выбираем один айсберг для анализа его окружения и извлекаем основную информацию из него
id = 174
test = iceberg_all.loc[id]
buysell = iceberg_all.loc[id][1]
date = iceberg_all.loc[id][2]
orderno = iceberg_all.loc[id][4]
price = iceberg_all.loc[id][5]
ticker = iceberg_all.loc[id][6]
time_place = iceberg_all.loc[id][7]
time_ident = iceberg_all.loc[id][8]
volume = iceberg_all.loc[id][10]

test

index                                  0
BUYSELL                                B
DATE                            20190304
ICEBERG                                1
ORDERNO                            47392
PRICE                             206.99
SECCODE                             SBER
TIME          2019-03-04 10:00:03.463857
TIME_IDENT    2019-03-04 10:00:03.463857
VOLUME                              -920
VOLUME_INI                          1000
DELTA                    0 days 00:00:00
Name: 174, dtype: object

In [185]:
%%time
cur_price = orders_instr.loc[ice_index,'PRICE']
ice_index = orders_instr.index[(orders_instr.ORDERNO == orderno)&(orders_instr.ACTION == 2)&(orders_instr.TIME == time_place)].to_list()[0]

for i in range(ice_index):
        
        cur_index = ice_index - i
        for_lob = orders_instr[orders_instr.index <= cur_index]

        if buysell == 'B':
            
            if GlassBuild(for_lob)[0] > cur_price:

                left_border = orders_instr.loc[cur_index,'TIME']
                left_index = cur_index
                
                break

        elif buysell == 'S':
            
            if GlassBuild(for_lob)[2] < cur_price:
                
                left_border = orders_instr.loc[cur_index,'TIME']
                left_index = cur_index
                 
                break

print(left_border, left_index)

2019-03-04 10:00:03.021387 3993
CPU times: user 3.1 s, sys: 43.6 ms, total: 3.15 s
Wall time: 3.23 s


# (16.7) Отбираем все сделки, которые попали в найденный интервал, в котором лучшая цена была на уровне цены айсберга

In [186]:
# сначала смотрим вообще на все, что попало в интервал до выставления айсберг-заявки
orders_instr[(orders_instr.TIME >= left_border) & (orders_instr.TIME <= time_place)]

,index,NO,SECCODE,BUYSELL,TIME,ORDERNO,ACTION,PRICE,VOLUME,TRADENO,TRADEPRICE
3991,47824,47825,SBER,B,2019-03-04 10:00:03.021387,46640,1,207.01,150,NaN,NaN
3992,47825,47826,SBER,B,2019-03-04 10:00:03.021387,46640,2,207.01,40,2.935625e+09,206.93
3993,47826,47827,SBER,S,2019-03-04 10:00:03.021387,44080,2,206.93,40,2.935625e+09,206.93
3994,47827,47828,SBER,B,2019-03-04 10:00:03.021387,46640,2,207.01,110,2.935625e+09,206.93
3995,47828,47829,SBER,S,2019-03-04 10:00:03.021387,44083,2,206.93,110,2.935625e+09,206.93
...,...,...,...,...,...,...,...,...,...,...,...
4243,50256,50257,SBER,B,2019-03-04 10:00:03.898113,48439,1,207.01,430,NaN,NaN
4244,50257,50258,SBER,B,2019-03-04 10:00:03.898113,48439,2,207.01,420,2.935625e+09,206.98
4245,50258,50259,SBER,S,2019-03-04 10:00:03.898113,48068,2,206.98,420,2.935625e+09,206.98
4246,50259,50260,SBER,B,2019-03-04 10:00:03.898113,48439,2,207.01,10,2.935625e+09,206.99


In [189]:
# сначала смотрим вообще на все, что попало в интервал после вскрытия айсберг-заявки
orders_instr[(orders_instr.TIME >= time_ident) & (orders_instr.TIME <= right_border)]

,index,NO,SECCODE,BUYSELL,TIME,ORDERNO,ACTION,PRICE,VOLUME,TRADENO,TRADEPRICE
4060,48773,48774,SBER,B,2019-03-04 10:00:03.463857,47392,1,206.99,1000,NaN,NaN
4061,48774,48775,SBER,B,2019-03-04 10:00:03.463857,47392,2,206.99,1920,2.935625e+09,206.90
4062,48775,48776,SBER,S,2019-03-04 10:00:03.463857,46818,2,206.90,1920,2.935625e+09,206.90
4063,48776,48777,SBER,B,2019-03-04 10:00:03.463857,47393,1,206.99,80,NaN,NaN
4064,48777,48778,SBER,B,2019-03-04 10:00:03.463857,47393,2,206.99,200,2.935625e+09,206.90
...,...,...,...,...,...,...,...,...,...,...,...
4243,50256,50257,SBER,B,2019-03-04 10:00:03.898113,48439,1,207.01,430,NaN,NaN
4244,50257,50258,SBER,B,2019-03-04 10:00:03.898113,48439,2,207.01,420,2.935625e+09,206.98
4245,50258,50259,SBER,S,2019-03-04 10:00:03.898113,48068,2,206.98,420,2.935625e+09,206.98
4246,50259,50260,SBER,B,2019-03-04 10:00:03.898113,48439,2,207.01,10,2.935625e+09,206.99


In [173]:
iceberg_all.iloc[174]

index                                  0
BUYSELL                                B
DATE                            20190304
ICEBERG                                1
ORDERNO                            47392
PRICE                             206.99
SECCODE                             SBER
TIME          2019-03-04 10:00:03.463857
TIME_IDENT    2019-03-04 10:00:03.463857
VOLUME                              -920
VOLUME_INI                          1000
DELTA                    0 days 00:00:00
Name: 174, dtype: object